In [33]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from xgboost import XGBRegressor

In [34]:
data = pd.read_csv('aggr.csv')

In [35]:
data

,Unnamed: 0,filename,mean,std,max,median,freq
0,0,спокойное дыхание пульс 90-Обрезка 14.MOV.csv,66.686331,11.800362,4447.066748,66.686331,0.0
1,0,спокойное дыхание пульс 90-Обрезка 15.MOV.csv,66.686331,11.800362,4447.066748,66.686331,0.0
2,0,спокойное дыхание пульс 90-Обрезка 21.MOV.csv,66.686331,11.800362,4447.066748,66.686331,0.0
3,0,спокойное дыхание пульс 90-Обрезка 02.MOV.csv,85.626944,13.632683,7331.973539,85.626944,0.0
4,0,спокойное дыхание пульс 90-Обрезка 12.MOV.csv,66.686331,11.800362,4447.066748,66.686331,0.0
...,...,...,...,...,...,...,...
69,0,пульс 116-90-Обрезка 04.MOV.csv,57.397457,10.821405,3294.468075,57.397457,0.0
70,0,пульс 116-90-Обрезка 13.MOV.csv,60.340288,11.138351,3640.950307,60.340288,0.0
71,0,пульс 116-90-Обрезка 14.MOV.csv,77.478463,12.867879,6002.912217,77.478463,0.0
72,0,пульс 116-90-Обрезка 18.MOV.csv,46.993063,9.641280,2208.347992,46.993063,0.0


In [36]:
for i in data.filename:
    print(i)

спокойное дыхание пульс 90-Обрезка 14.MOV.csv
спокойное дыхание пульс 90-Обрезка 15.MOV.csv
спокойное дыхание пульс 90-Обрезка 21.MOV.csv
спокойное дыхание пульс 90-Обрезка 02.MOV.csv
спокойное дыхание пульс 90-Обрезка 12.MOV.csv
спокойное дыхание пульс 90-Обрезка 03.MOV.csv
спокойное дыхание пульс 90-Обрезка 17.MOV.csv
спокойное дыхание пульс 90-Обрезка 22.MOV.csv
спокойное дыхание пульс 90-Обрезка 09.MOV.csv
спокойное дыхание пульс 90-Обрезка 20.MOV.csv
спокойное дыхание пульс 90-Обрезка 18.MOV.csv
спокойное дыхание пульс 90-Обрезка 11.MOV.csv
спокойное дыхание пульс 90-Обрезка 04.MOV.csv
спокойное дыхание пульс 90-Обрезка 05.MOV.csv
спокойное дыхание пульс 90-Обрезка 06.MOV.csv
спокойное дыхание пульс 90-Обрезка 19.MOV.csv
спокойное дыхание пульс 90-Обрезка 07.MOV.csv
спокойное дыхание пульс 90-Обрезка 13.MOV.csv
спокойное дыхание пульс 90-Обрезка 16.MOV.csv
пульс 160-150 4 шаблон-Обрезка 10.MOV.csv
пульс 160-150 4 шаблон-Обрезка 02.MOV.csv
пульс 160-150 4 шаблон-Обрезка 05.MOV.csv


In [37]:
a = []
for i in data.filename:
    if 'delts3' in i:
        continue
    if 'пульс 116-90' in i:
        if int(i[-10]) * 10 + int(i[-9]) <= 9:
            a.append(116)
        elif (int(i[-10]) * 10 + int(i[-9]) > 9) and (int(i[-10]) * 10 + int(i[-9]) <= 11):
            a.append(100)
        elif (int(i[-10]) * 10 + int(i[-9]) >= 12) and (int(i[-10]) * 10 + int(i[-9]) <= 14):
            a.append(93)   
        else:
            a.append(90)  
    elif 'спокойное дыхание пульс 90' in i:
        if int(i[-10]) * 10 + int(i[-9]) <= 5:
            a.append(90)
        elif (int(i[-10]) * 10 + int(i[-9]) == 6) :
            a.append(87)
        elif (int(i[-10]) * 10 + int(i[-9]) > 6) and (int(i[-10]) * 10 + int(i[-9]) <= 17):
            a.append(95)   
        else:
            a.append(100)
    elif 'пульс 160-150 4 шаблон' in i:
        if int(i[-10]) * 10 + int(i[-9]) <= 16:
            a.append(160)
        else:
            a.append(150)
    elif 'Бутербор17' in i:
        a.append(70)
    elif 'пульс 90-80 видео по 2 шаблону' in i:
        if int(i[-10]) * 10 + int(i[-9]) <= 12:
            a.append(92)
        elif (int(i[-10]) * 10 + int(i[-9]) > 12) and (int(i[-10]) * 10 + int(i[-9]) <= 16):
            a.append(80) 
        else:
            a.append(86)
    else:
        a.append(86)

In [38]:
data = data.loc[data.filename != 'delts3.csv']
data['y'] = a

<ipython-input-38-d67f1591679f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['y'] = a


In [39]:
data = data.drop(data.loc[:, data.columns.str.contains('^Unnamed')],axis = 1)
data = data.drop(['filename'],axis = 1)

In [40]:
X = data.drop(['y'], axis = 1)
y = data['y']

In [41]:
X


,mean,std,max,median,freq
0,66.686331,11.800362,4447.066748,66.686331,0.0
1,66.686331,11.800362,4447.066748,66.686331,0.0
2,66.686331,11.800362,4447.066748,66.686331,0.0
3,85.626944,13.632683,7331.973539,85.626944,0.0
4,66.686331,11.800362,4447.066748,66.686331,0.0
...,...,...,...,...,...
69,57.397457,10.821405,3294.468075,57.397457,0.0
70,60.340288,11.138351,3640.950307,60.340288,0.0
71,77.478463,12.867879,6002.912217,77.478463,0.0
72,46.993063,9.641280,2208.347992,46.993063,0.0


In [42]:
y

0      95
1      95
2     100
3      90
4      95
     ... 
69    116
70     93
71     93
72     90
73    116
Name: y, Length: 73, dtype: int64

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [44]:
xgb = XGBRegressor(task_type = 'GPU', iterations = 500)

In [45]:
xgb.fit(X_train, y_train)

[08:27:24] WARNING: ../src/learner.cc:541: 
Parameters: { iterations, task_type } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='', iterations=500,
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             task_type='GPU', tree_method='exact', validate_parameters=1,
             verbosity=None)

In [46]:
y_pred = xgb.predict(X_test)

In [47]:
y_test

50     80
54     86
22    160
44     92
29     70
26    160
71     93
51     86
35     92
7     100
43     80
56     86
41     92
61    100
34     92
66     90
28     70
72     90
57    116
53     86
6      95
49     86
Name: y, dtype: int64

In [48]:
pd.Series(y_pred)

0      88.000763
1      89.999771
2     160.000168
3      88.000763
4     115.997650
5     159.998062
6      93.000938
7      88.000763
8      89.999771
9      94.812607
10    106.830544
11     88.000763
12     94.812607
13     77.999443
14     94.812607
15    115.998184
16     77.999443
17    115.998184
18     93.000938
19     70.000244
20     94.812607
21     89.999771
dtype: float32

In [49]:
abs(y_pred - y_test)

50     8.000763
54     3.999771
22     0.000168
44     3.999237
29    45.997650
26     0.001938
71     0.000938
51     2.000763
35     2.000229
7      5.187393
43    26.830544
56     2.000763
41     2.812607
61    22.000557
34     2.812607
66    25.998184
28     7.999443
72    25.998184
57    22.999062
53    15.999756
6      0.187393
49     3.999771
Name: y, dtype: float64

In [50]:
r2_score(y_pred, y_test)

0.48096941725776354

In [51]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,mean,std,max,median,freq,y
mean,1.000000,1.000000,1.000000,1.000000,-0.041345,-0.039051
std,1.000000,1.000000,1.000000,1.000000,-0.041345,-0.039051
max,1.000000,1.000000,1.000000,1.000000,-0.041345,-0.039051
median,1.000000,1.000000,1.000000,1.000000,-0.041345,-0.039051
freq,-0.041345,-0.041345,-0.041345,-0.041345,1.000000,0.897900
y,-0.039051,-0.039051,-0.039051,-0.039051,0.897900,1.000000


In [52]:
xgb.save_model('xgb_model.json')